In [14]:
import os, pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
%matplotlib inline
%config InlineBackend.figure_format='retina'

import warnings
warnings.filterwarnings("ignore")
def euc_dist(x, y): return euclidean_distances([x],[y])[0][0]

In [37]:
dataset_path = {
    'prostatex': '/net/scratch/hanliu/radiology/prostatex/tbakd3_npy/5folds/4/embs',
    # 'prostatex': '/net/scratch/hanliu-shared/tbakd3_npy/5folds/4/embs',
    'bm': '/net/scratch/hanliu/radiology/explain_teach/data/bm/embs',
}
dataset = 'bm'
model = 'resnt'
name = 'emb10'
train_path = '{}/{}_train_{}.pkl'.format(dataset_path[dataset], model, name)
valid_path = '{}/{}_valid_{}.pkl'.format(dataset_path[dataset], model, name)
f_train, _, y_train, X_train = pickle.load(open(train_path, "rb"))
f_valid, _, y_valid, X_valid = pickle.load(open(valid_path, "rb"))
resnt_train = X_train
# dwac_train = X_train
# dwac10_train = X_train

In [38]:
rt_res = []
d2_res = []
d10_res = []

In [40]:
dist = pickle.load(open("embeds/lpips.bm.train.pkl", "rb"))

In [41]:
np.random.seed(42)
for i in range(10000):
    a, p, n = np.random.choice(len(dist), 3, replace=False)
    ap = dist[a, p] < dist[a, n]
    rd = euc_dist(resnt_train[a], resnt_train[p]) < euc_dist(resnt_train[a], resnt_train[n])
    d2d = euc_dist(dwac_train[a], dwac_train[p]) < euc_dist(dwac_train[a], dwac_train[n])
    d10d = euc_dist(dwac10_train[a], dwac10_train[p]) < euc_dist(dwac10_train[a], dwac10_train[n])
    rt_res.append(ap == rd)
    d2_res.append(ap == d2d)
    d10_res.append(ap == d10d)


In [42]:
np.mean(rt_res), np.mean(d2_res), np.mean(d10_res)

(0.7016, 0.5334, 0.5366)

In [43]:
from scipy import stats
def get_ci(samples, confidence=0.95):
    return 2 * stats.sem(samples) * stats.t.ppf((1 + confidence) / 2., len(samples)-1)

In [44]:
get_ci(rt_res), get_ci(d2_res), get_ci(d10_res)

(0.017938931680176674, 0.019559207281043055, 0.019550403499102146)

In [12]:
a,p,n = np.random.choice(len(dist), 3, replace=False)

In [17]:
def euc_dist(x, y): return np.sqrt(np.dot(x, x) - 2 * np.dot(x, y) + np.dot(y, y))